In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import re
import time
import pandas as pd


In [ ]:
def page_down(driver):
    for _ in range(10):
        body = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd')
        body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

def separate_to_grade(places, location):
        return places

def url_parse(url):
    parse = re.search(r'@([-+]?\d*\.\d+|\d+),([-+]?\d*\.\d+|\d+)', url)
    # parse = re.search(r'!3d([\d.]+)!4d([\d.]+)', url)   
    if parse:
        lat = float(parse.group(1))
        lon = float(parse.group(2))
        return lat, lon
    else:
        return None


In [ ]:
driver = webdriver.Chrome()
# parse = re.search(r'!3d([\d.]+)!4d([\d.]+)', url)
# 1. 구글 지도 접속
driver.get('https://www.google.com/maps/?entry=ttu')
time.sleep(2)

driver.find_element(By.CSS_SELECTOR, "input#searchboxinput").send_keys('오사카 명소') # 일본 지역 크롤링 후 format으로 바꿔서 사용
driver.find_element(By.CSS_SELECTOR, "button#searchbox-searchbutton").click()
time.sleep(2)

page_down(driver)


In [ ]:
places = []
dataset = [0, 1, 2]
location = []
url_lst = []
count = 1
append_count = 0
url = ''

In [ ]:
while True:
    print(count)
    places_value = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div:nth-child({})'.format(count))
    if places_value.text == '마지막 항목입니다.':
        break

    # 예외 처리를 사용한 이유 : 
    # 각 div 태그의 순서가 달라 nth-child()를 사용할 수 없음
    # 이후 조금 더 효율적인 방법을 찾아야함
    try:
        driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div:nth-child({}) > div > a'.format(count)).click()
        time.sleep(4)

        location.append(driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.Hu9e2e.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf > div:nth-child(11) > div:nth-child(3) > button > div > div.rogA2c > div.Io6YTe.fontBodyMedium.kR99db').text)
        driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.Hu9e2e.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.BHymgf.eiJcBe.bJUD0c > div > div > div:nth-child(3) > span > button').click()
        time.sleep(2)
    except:
        location.append('None')
        
    places_value = places_value.text.split('\n')
    print(places_value)
    places.append([])
    for i in dataset:
        places[append_count].append(places_value[i])
    append_count += 1
    count += 2


In [ ]:
for i in range(len(places)):
        places[i][1] = places[i][1].replace('(', ', ').replace(')', '') # 추가
        places[i].append(location[i])

In [ ]:
places

In [ ]:
df = pd.DataFrame(places, columns=['name', 'score', 'category', 'location', 'lat', 'lon'])
df

In [ ]:
df.to_csv('osaka_places.csv', index=False)

In [ ]:
driver.quit()
